In [1]:
import pandas as pd

from selenium import webdriver

import warnings
warnings.filterwarnings('ignore')

from selenium.webdriver.common.by import By

from selenium.webdriver.chrome.options import Options

import concurrent.futures 

from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException

from bs4 import BeautifulSoup as bs
import requests as req

import time

In [2]:
#Cookies 
options=Options()
options.add_argument(r'user-data-dir=C:\Users\matia\anaconda3\Lib\site-packages\selenium\cookies') #Cargamos las cookies

In [3]:
url = 'https://www.transfermarkt.es/schnellsuche/ergebnis/schnellsuche?query=abner'

PATH = 'driver/chromedriver.exe'

In [4]:
driver = webdriver.Chrome(PATH, options=options)

driver.get(url)

In [5]:
#Scrapeo para sacar el valor de un jugador
driver.find_elements(By.CSS_SELECTOR, '#yw0 > table > tbody > tr:nth-child(1) > td.rechts.hauptlink')[0].text

'7,00 mill. €'

In [6]:
#Hago una funcion para scrapear el valor de todos mis jugadores
def valor(futbolista,PATH):
    
    driver = webdriver.Chrome(PATH) 
    time.sleep(2)
    driver.get(f'https://www.transfermarkt.es/schnellsuche/ergebnis/schnellsuche?query={futbolista}')
    time.sleep(2)
    try:
        valor = driver.find_elements(By.CSS_SELECTOR, '#yw0 > table > tbody > tr:nth-child(1) > td.rechts.hauptlink')[0].text
    except:
        valor = 'NaN'
        print(f'No se encontro el valor de {futbolista}')

    driver.quit()
    return valor


In [7]:
#Pruebo la funcion
valor('Carvajal',PATH)

'12,00 mill. €'

# Lo aplico a mi dataframe


In [8]:
df = pd.read_csv('../../csv/csv_limpios/jugadores_2022_lp.csv')

In [9]:
lista_jugadores_22 = df['Jugador'].tolist()

In [10]:
with concurrent.futures.ThreadPoolExecutor(max_workers=12) as executor:  # Limita el número de hilos  
    futures = [executor.submit(valor, jugador, PATH) for jugador in lista_jugadores_22]
    resultados = [f.result() for f in concurrent.futures.as_completed(futures)]

In [11]:
df['Valor Mercado'] = resultados

# Lo aplico a mi dataframe de 2023


In [12]:
df2 = pd.read_csv('../../csv/csv_limpios/jugadores_2023_lp.csv')

In [13]:
lista_jugadores_23 = df2['Jugador'].tolist()
len(lista_jugadores_23)

470

In [14]:
with concurrent.futures.ThreadPoolExecutor(max_workers=12) as executor:  # Limita el número de hilos  
    futures = [executor.submit(valor, jugador, PATH) for jugador in lista_jugadores_23]
    resultados_23 = [f.result() for f in concurrent.futures.as_completed(futures)]

In [15]:
df2['Valor Mercado'] = resultados_23

In [16]:
df2

,Unnamed: 0,Jugador,País,Posc,Equipo,Edad,Nacimiento,PJ,Titular,Mín,Gls.,Asistencias,Goles+Asistencias,Tarjetas Amarillas,Tarjetas Rojas,Goles Esperados,Asistencias Esperadas,Valor Mercado
0,0,Abner,BRA,Defensa,Betis,23,2000,9,5,522,0,0,0,2,0,0.1,0.4,"2,50 mill. €"
1,1,Abdel Abqar,MAR,Defensa,Alavés,24,1999,11,11,989,0,0,0,3,0,0.2,0.0,"8,00 mill. €"
2,2,Marcos Acuña,ARG,Defensa,Sevilla,32,1991,7,6,411,0,0,0,2,0,0.1,0.3,"3,50 mill. €"
3,3,Joseph Aidoo,GHA,Defensa,Celta Vigo,28,1995,6,4,407,0,0,0,0,0,0.1,0.1,"3,00 mill. €"
4,4,Ilias Akhomach,ESP,Centrocampista,Villarreal,19,2004,10,2,346,0,1,1,2,0,0.1,0.4,"1,20 mill. €"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
465,474,Arsen Zakharyan,RUS,Centrocampista,Real Sociedad,20,2003,8,2,207,0,0,0,0,0,0.1,0.4,"5,00 mill. €"
466,475,Joseba Zaldúa,ESP,Defensa,Cádiz,31,1992,2,2,139,0,0,0,0,0,0.0,0.0,"40,00 mill. €"
467,476,Bryan Zaragoza,ESP,Centrocampista,Granada,22,2001,13,10,937,5,2,7,5,0,2.7,2.0,"18,00 mill. €"
468,477,Igor Zubeldia,ESP,Defensa,Real Sociedad,26,1997,13,12,1081,0,0,0,6,0,0.2,0.6,"1,50 mill. €"


# Exporto a csv limpio

In [17]:
df2.to_csv('../../csv/csv_limpios/jugadores_2023_lp.csv', index=False)